In [1]:
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from lohrasb.best_estimator import BaseModel
from sklearn.pipeline import Pipeline
from feature_engine.imputation import (
    CategoricalImputer,
    MeanMedianImputer
    )
from category_encoders import OrdinalEncoder
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score)
from sklearn.metrics import f1_score, make_scorer
from xgboost import *
import optuna
from lohrasb import logger


2023-07-02 13:05:56,943 :: matplotlib :: matplotlib data path: /Users/hjavedani/Documents/Lohrasb/.venv/lib/python3.10/site-packages/matplotlib/mpl-data
2023-07-02 13:05:56,949 :: matplotlib :: CONFIGDIR=/Users/hjavedani/.matplotlib
2023-07-02 13:05:56,952 :: matplotlib :: interactive is False
2023-07-02 13:05:56,953 :: matplotlib :: platform is darwin
2023-07-02 13:05:56,967 :: graphviz._tools :: deprecate positional args: graphviz.backend.piping.pipe(['renderer', 'formatter', 'neato_no_op', 'quiet'])
2023-07-02 13:05:56,969 :: graphviz._tools :: deprecate positional args: graphviz.backend.rendering.render(['renderer', 'formatter', 'neato_no_op', 'quiet'])
2023-07-02 13:05:56,971 :: graphviz._tools :: deprecate positional args: graphviz.backend.unflattening.unflatten(['stagger', 'fanout', 'chain', 'encoding'])
2023-07-02 13:05:56,972 :: graphviz._tools :: deprecate positional args: graphviz.backend.viewing.view(['quiet'])
2023-07-02 13:05:56,976 :: graphviz._tools :: deprecate positio

#### Example: Use Adult Data Set (a classification problem)
  
https://archive.ics.uci.edu/ml/datasets/Adult

#### Part 1: Use BestModel in sklearn pipeline


In [2]:
urldata= "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
# column names
col_names=["age", "workclass", "fnlwgt" , "education" ,"education-num",
"marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week",
"native-country","label"
]
# read data
data = pd.read_csv(urldata,header=None,names=col_names,sep=',')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


#### Define labels


In [3]:
data.loc[data['label']=='<=50K','label']=0
data.loc[data['label']==' <=50K','label']=0

data.loc[data['label']=='>50K','label']=1
data.loc[data['label']==' >50K','label']=1

data['label']=data['label'].astype(int)

#### Train test split

In [4]:
X = data.loc[:, data.columns != "label"]
y = data.loc[:, data.columns == "label"]


X_train, X_test, y_train, y_test =train_test_split(X, y, \
     test_size=0.33, stratify=y['label'], random_state=42)


#### Find feature types for later use

In [5]:
int_cols =  X_train.select_dtypes(include=['int']).columns.tolist()
float_cols =  X_train.select_dtypes(include=['float']).columns.tolist()
cat_cols =  X_train.select_dtypes(include=['object']).columns.tolist()


#### Define estimator and set its arguments 


In [6]:
estimator = XGBClassifier()
estimator_params = {
        "booster": optuna.distributions.CategoricalDistribution(choices=("gbtree","dart")),
        "max_depth": optuna.distributions.IntDistribution(10, 15),
        "gamma": optuna.distributions.FloatDistribution(0.5, 1.2, log=True),
        "subsample": optuna.distributions.FloatDistribution(0.8, 1.0)
        }

    

In [7]:
kwargs = {  # params for fit method or fit_params 
            'fit_newoptuna_kwargs' :{
            'sample_weight':None,
            },
            # params for GridSearchCV 
            'newoptuna_search_kwargs' : {
            'estimator':estimator,
            'param_distributions':estimator_params,
            'scoring' :'f1',
            'verbose':3,
            'n_jobs':-1,
            'cv':KFold(2),
            }
            }


In [8]:

obj = BaseModel().optimize_by_optunasearchcv(
        kwargs=kwargs    
        )

#### Build sklearn pipeline

In [9]:


pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model
            ('obj', obj)

 ])


#### Run Pipeline

In [10]:
pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)


/Users/hjavedani/Documents/Lohrasb/lohrasb/base_classes/optimizer_bases.py:554: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  self.__newoptuna_search = OptunaSearchCV(**self.newoptuna_search_kwargs)
[I 2023-07-02 13:05:58,918] A new study created in memory with name: no-name-efc0b173-ede1-4842-962e-2722b2d28cb4
[I 2023-07-02 13:05:58,919] Searching the best hyperparameters using 21815 samples...
[I 2023-07-02 13:06:09,878] Trial 2 finished with value: 0.6731805554403802 and parameters: {'booster': 'gbtree', 'max_depth': 10, 'gamma': 0.9698549327904533, 'subsample': 0.9061241274735645}. Best is trial 2 with value: 0.6731805554403802.
[I 2023-07-02 13:06:10,066] Trial 6 finished with value: 0.6742600479596783 and parameters: {'booster': 'gbtree', 'max_depth': 10, 'gamma': 0.7207882005474862, 'subsample': 0.8402692877276656}. Best is trial 6 with value: 0.6742600479596783.
[I 2023-07-02 13:06:12,349] Trial 1 finishe

#### Check performance of the pipeline

In [11]:
print('F1 score : ')
print(f1_score(y_test,y_pred))
print('Classification report : ')
print(classification_report(y_test,y_pred))
print('Confusion matrix : ')
print(confusion_matrix(y_test,y_pred))


F1 score : 
0.6827358879274825
Classification report : 
              precision    recall  f1-score   support

           0       0.89      0.93      0.91      8158
           1       0.73      0.64      0.68      2588

    accuracy                           0.86     10746
   macro avg       0.81      0.78      0.80     10746
weighted avg       0.85      0.86      0.85     10746

Confusion matrix : 
[[7549  609]
 [ 931 1657]]


#### Part 2:  Use BestModel as a standalone estimator 

In [12]:
X_train, X_test, y_train, y_test =train_test_split(X, y, \
     test_size=0.33, stratify=y['label'], random_state=42)

#### Transform features to make them ready for model input

In [13]:
transform_pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model

 ])

#### Transform X_train and X_test

In [14]:
X_train=transform_pipeline.fit_transform(X_train,y_train)
X_test=transform_pipeline.transform(X_test)


#### Train model and predict

In [15]:
obj.fit(X_train,y_train)
y_pred = obj.predict(X_test)

/Users/hjavedani/Documents/Lohrasb/lohrasb/base_classes/optimizer_bases.py:554: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  self.__newoptuna_search = OptunaSearchCV(**self.newoptuna_search_kwargs)
[I 2023-07-02 13:06:32,233] A new study created in memory with name: no-name-bec18ce3-f65b-4fa7-a611-fe189d1b3bc1
[I 2023-07-02 13:06:32,233] Searching the best hyperparameters using 21815 samples...
[I 2023-07-02 13:06:45,399] Trial 5 finished with value: 0.6717867018986229 and parameters: {'booster': 'gbtree', 'max_depth': 10, 'gamma': 0.523235091341924, 'subsample': 0.8443471704411979}. Best is trial 5 with value: 0.6717867018986229.
[I 2023-07-02 13:06:49,413] Trial 2 finished with value: 0.6712871315895095 and parameters: {'booster': 'gbtree', 'max_depth': 13, 'gamma': 0.7597514874990623, 'subsample': 0.8817769134272618}. Best is trial 5 with value: 0.6717867018986229.
[I 2023-07-02 13:06:50,998] Trial 7 finished

#### Check performance of the pipeline

In [16]:
print('F1 score : ')
print(f1_score(y_test,y_pred))
print('Classification report : ')
print(classification_report(y_test,y_pred))
print('Confusion matrix : ')
print(confusion_matrix(y_test,y_pred))

F1 score : 
0.6959721938253937
Classification report : 
              precision    recall  f1-score   support

           0       0.90      0.93      0.91      8158
           1       0.74      0.66      0.70      2588

    accuracy                           0.86     10746
   macro avg       0.82      0.79      0.80     10746
weighted avg       0.86      0.86      0.86     10746

Confusion matrix : 
[[7557  601]
 [ 886 1702]]


In [17]:
obj.get_best_estimator()

XGBClassifier(base_score=0.5, booster='dart', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.6757806456611722,
              gpu_id=-1, grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=10, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [18]:
obj.best_estimator

XGBClassifier(base_score=0.5, booster='dart', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.6757806456611722,
              gpu_id=-1, grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=10, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)

#### Get fitted grid search object and its attributes

In [19]:
NewSearchObj = obj.get_optimized_object()

In [20]:
NewSearchObj

OptunaSearchCV(cv=KFold(n_splits=2, random_state=None, shuffle=False),
               estimator=XGBClassifier(base_score=None, booster=None,
                                       callbacks=None, colsample_bylevel=None,
                                       colsample_bynode=None,
                                       colsample_bytree=None,
                                       early_stopping_rounds=None,
                                       enable_categorical=False,
                                       eval_metric=None, feature_types=None,
                                       gamma=None, gpu_id=None,
                                       grow_policy=None, importance_type=Non...
                                       num_parallel_tree=None, predictor=None,
                                       random_state=None, ...),
               n_jobs=-1,
               param_distributions={'booster': CategoricalDistribution(choices=('gbtree', 'dart')),
                                    'gamma': FloatDistribution(high=1.2, log=True, low=0.5, step=None),
                                    'max_depth': IntDistribution(high=15, log=False, low=10, step=1),
                                    'subsample': FloatDistribution(high=1.0, log=False, low=0.8, step=None)},
               scoring='f1', verbose=3)